In [1]:
import numpy as np
from cellpose import models, core, io
from spotiflow.model import Spotiflow
from pathlib import Path
import os
import apoc
from tqdm import tqdm
import pyclesperanto_prototype as cle 
import pandas as pd
from utils import list_images, read_image, brightfield_correction, detect_infection_load, extract_intensity_information, puncta_detection

io.logger_setup() # run this to get printing of progress

#Check if notebook has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

#Activate .cle GPU acceleration
cle.select_device("RTX")

#Load pre-trained Cellpose-SAM and Spotiflow models
model = models.CellposeModel(gpu=True)
spotiflow_model = Spotiflow.from_pretrained("general")



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




c:\Users\adiez_cmic\miniforge3\envs\hcs_cellpose\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-10-03 10:48:04,154 [INFO] WRITING LOG OUTPUT TO C:\Users\adiez_cmic\.cellpose\run.log
2025-10-03 10:48:04,154 [INFO] 
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0
2025-10-03 10:48:04,208 [INFO] ** TORCH CUDA version installed and working. **
2025-10-03 10:48:04,208 [INFO] ** TORCH CUDA version installed and working. **
2025-10-03 10:48:04,208 [INFO] >>>> using GPU (CUDA)
2025-10-03 10:48:05,081 [INFO] >>>> loading model C:\Users\adiez_cmic\.cellpose\models\cpsam
INFO:spotiflow.model.spotiflow:Loading pretrained model: general
2025-10-03 10:48:05,509 [INFO] Loading pretrained model: general


In [2]:
# Load pretrained Object Classifier for Mycobacterium infection detection
mtb_cl_filename = "./raw_data/SD_DAPI_Mtb_detection_training/Mtb_segmenter.cl"
mtb_segmenter = apoc.ObjectSegmenter(opencl_filename=mtb_cl_filename)

# Create a list of subdirectories containing the Nuc string (stained for nuclei)
main_directory_path = Path("X:\Lisa\siMtb screen I_LØ")

folders_with_nuc = [
    name for name in os.listdir(main_directory_path)
    if os.path.isdir(os.path.join(main_directory_path, name)) and "Nuc" in name
]

print(folders_with_nuc)

['Plate 01_Nuc', 'Plate 02_Nuc', 'Plate 03_Nuc', 'Plate 04_Nuc', 'Plate 05_Nuc', 'Plate 06_Nuc', 'Plate 07_Nuc', 'Plate 08_Nuc', 'Plate 09_Nuc', 'Plate 10_Nuc', 'Plate 11_Nuc', 'Plate 12_Nuc', 'Plate 13_Nuc', 'Plate 17_Nuc', 'Plate 14_Nuc', 'Plate 15_Nuc', 'Plate 16_Nuc']


In [3]:
# Define the channels you want to analyze using the following structure:
# markers = [(channel_name, channel_nr),(..., ...)]
# Remember in Python one starts counting from 0, so your first channel will be 0
# i.e. markers = [("ki67", 0), ("neun", 1)]

# This list hold the markers for signal intensity analysis 
markers = [("LC3B", 0), ("GAL3", 1), ("Chmp4B", 2), ("Mtb", 3)]

# This list hold the markers for spot (puncta) detection analysis 
puncta_markers = [("LC3B", 0), ("GAL3", 1), ("Chmp4B", 2)]

In [ ]:
for folder in tqdm(folders_with_nuc):

    print(f"Analyzing Plate: {folder}")

    # Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
    directory_path = Path(f"X:\Lisa\siMtb screen I_LØ\{folder}")

    # Iterate through the .czi and .nd2 files in the directory
    images = list_images(directory_path)

    # Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
    slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)

    # Substract uneven and remove background from BF by obtaining the median of all BF channels
    bf_correction = brightfield_correction(directory_path, images, slicing_factor_xy) 

    # Empty list to populate with per well features
    per_well_props = []

    # Empty list to populate with per infection stats
    infection_stats = []

    # Loop through well images in each plate folder (A1 --> H12)
    for image in tqdm(images):

        # Read image, apply slicing if needed and return filename and img as a np array
        img, filename = read_image(image, slicing_factor_xy)

        # Extract plate number and well_id
        plate_nr = filename.split("_")[0]
        well_id = filename.split("Wells-")[1].split("__")[0]

        # Predict cytoplasm labels using CellposeSAM
        cytoplasm_labels, flows, styles = model.eval(np.stack((img[[0,1]].sum(axis=0), (img[4] - bf_correction)), axis=0), niter=1000) # need to check the arguments

        # ----- Mtb infection detection ------

        infected_labels = detect_infection_load(img, mtb_segmenter, cytoplasm_labels, plate_nr, well_id, infection_stats)

        # ----- per_label intensity information ------

        props_df = extract_intensity_information(img, cytoplasm_labels, markers, plate_nr, well_id, image)

        # ----- puncta filtering and counting ------ 

        props_df = puncta_detection(img, puncta_markers, spotiflow_model, cytoplasm_labels, props_df)

        # ----- Dataframe update and save logic ------ 

        # Rename label(id) to CellProfiler format ObjectNumber
        props_df.rename(columns={"label": "ObjectNumber"}, inplace=True)

        # Add infected flag to props_df if labels is in infected_labels
        # Find position of "ObjectNumber" column
        col_idx = props_df.columns.get_loc("ObjectNumber")

        # Insert new column right after "ObjectNumber"
        props_df.insert(col_idx + 1, "Mtb_infected", props_df["ObjectNumber"].isin(infected_labels))

        # Append each props_df to per_well_props
        per_well_props.append(props_df)

    # ----- Data save after loop completion ------

    # Transform infection stats list into a dataframe to store it as .csv
    df = pd.DataFrame(infection_stats)
    df.to_csv(f"./results_infection_{plate_nr}.csv")

    # Concatenate all per_well_props into final_df
    final_df = pd.concat(per_well_props, ignore_index=True)
    final_df.to_csv(f"./results_per_label_{plate_nr}.csv")

  0%|          | 0/17 [00:00<?, ?it/s]

Analyzing Plate: Plate 01_Nuc



Image analyzed: Plate01_Nuc_Wells-A10__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR

Detecting infection load...
Total cells: 1493
Percentage infected:17.41
Extracting per marker intensity information...
Detecting spots in puncta markers...



Image analyzed: Plate01_Nuc_Wells-A11__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR

Detecting infection load...
Total cells: 1978
Percentage infected:23.66
Extracting per marker intensity information...
Detecting spots in puncta markers...



Image analyzed: Plate01_Nuc_Wells-A12__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR

Detecting infection load...
Total cells: 1913
Percentage infected:23.37
Extracting per marker intensity information...
Detecting spots in puncta markers...


...


Image analyzed: Plate16_Nuc_Wells-C1__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR

Detecting infection load...
Total cells: 1923
Percentage infected:23.3
Extracting per marker intensity information...
Detecting spots in puncta markers...



Image analyzed: Plate16_Nuc_Wells-C2__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR

Detecting infection load...
Total cells: 2022
Percentage infected:21.41
Extracting per marker intensity information...
Detecting spots in puncta markers...



Image analyzed: Plate16_Nuc_Wells-C3__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR

Detecting infection load...
Total cells: 1818
Percentage infected:19.75
Extracting per marker intensity information...
Detecting spots in puncta markers...



Image analyzed: Plate16_Nuc_Wells-C4__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR

Detecting infection load...
Total cells: 2201
Percentage infected:23.49
Extracting per marker intensity information...
Detecting spots in puncta markers...


100%|██████████| 17/17 [13:33:50<00:00, 2872.37s/it]
